# Using energy flow masks in the Unet model

## Idea

Cellpose is a software to segment object with irregular shapes, such as cells, but is also general and work for any image with similar objects such as rocks or shells https://www.biorxiv.org/content/10.1101/2020.02.02.931238v1.full.pdf.

To create this software they have used a Unet model architechture. As labels they have converted regular mask images with one pixel value per object to energy flow maps. Creating a heat diffusion simulation starting at the center of the cell expanding to the edges. 

My idea is to use the Unet we have, and try to implement the labels used in the cellpose model so that we would get a model specialized on our dataset. I think that the main thing we need to change for it to work is the loss function, and then do some tweaks in the model.


## Current model



The model I am using now is the Unet originally created by Broad Institute (BBBC) for nuclei segmentation https://www.biorxiv.org/content/10.1101/335216v2.full.pdf

The architecture of the model can be found here https://github.com/Aitslab/imageanalysis/blob/master/Malou_dir/README_Folder/Nuclei_Segmentation_Project/2_Final_Models/1_Model1/Unet/utils/model_builder.py

The labels for this model are 3 class labels; background(red), boundary(blue) and interior(green).


images show:
**Cell channel**
1. Normalized input image
2. Raw annotation 
3. 3 class boundary label


## <center>Cell images</center>

<center>Normalized input image</center> | <center>Raw annotation </center>|<center> 3 class boundary label</center>
- | - | - 
<img src="images_jupyter/norm_images/MFGTMPcx7_170702000001_B14f07d1.png">| <img src="images_jupyter/raw_annotation/MFGTMPcx7_170702000001_B14f07d1.png">| <img src="images_jupyter/boundary_labels/MFGTMPcx7_170702000001_B14f07d1.png">


images show:
**Nuclei channel**
1. Normalized input image
2. Raw annotation 
3. 3 class boundary label

## <center>Nuclei images</center>

<center>Normalized input image</center> | <center>Raw annotation </center>|<center> 3 class boundary label</center>
- | - | - 
<img src="images_jupyter/norm_images/MFGTMPcx7_170702000001_B14f07d0.png">| <img src="images_jupyter/raw_annotation/MFGTMPcx7_170702000001_B14f07d0.png">| <img src="images_jupyter/boundary_labels/MFGTMPcx7_170702000001_B14f07d0.png">



### Loss function for BBBC unet

The current loss function looks like this, with softmax crossentropy loss function, and added penalty for the boundary class

```python
def weighted_crossentropy(y_true, y_pred):

    class_weights = tf.constant([[[[1., 1., 10.]]]])

    unweighted_losses = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)

    weights = tf.reduce_sum(class_weights * y_true, axis=-1)

    weighted_losses = weights * unweighted_losses

    loss = tf.reduce_mean(weighted_losses)

    return loss
```

## Cellpose approach

### Creating of energy flow labels

To create the energy flow masks, I have selected code from cellpose github page and converted the current masks (script provided at the end of this notebook, starting with the cellpose code, and my additional code to create my masks). 

I interpret it as this is the final masks to input to the model. the function ```flows_to_labels``` will take a list of images and create masks using the ```masks_to_flows```function. I manage to create the masks, looking as follows. What I am unsure about is if they are created correctly. Looking in the function  ```flows_to_labels``` on the following line; 
```python
veci = [masks_to_flows(labels[n][0])[0] for n in trange(nimg)]
```
this line will create a list of the flows, the indexing here will only take the first dimension value (1104 in the case of our images) of the images as input to the ```masks_to_flows``` function, which I am unsure about being correct. Although, it works and creates the following images (the merged image being the one I will use in the model):



images show:
1. Xflow image
2. Yflow image 
3. Cell probability label
4. Merged image

## <center>Flow images</center>

<center>X flow image</center> | <center>Y flow image </center>|<center> Cell probability image</center>|<center> Merged image </center>
- | - | - | -
<img src="images_jupyter/flows/Xflow/MFGTMPcx7_170702000001_B14f07d1.png">| <img src="images_jupyter/flows/Yflow/MFGTMPcx7_170702000001_B14f07d1.png">| <img src="images_jupyter/flows/Cellprob/MFGTMPcx7_170702000001_B14f07d1.png">| <img src="images_jupyter/flows/MFGTMPcx7_170702000001_B14f07d1.png">


### Image from Cellpose: a generalist algorithm for cellular segmentation paper describing the flows

<img src="images_jupyter/cellpose-img.png">

## Cellpose model


The Cellpose model is created with the python package mxnet, which is all new to me. I have looked at their code in their github page, and interpret the loss function. 

On their github I found two loss functions, they seem to combine several model architectures that I don't know how it works. 

The loss function I tried to convert looks like this: 

```python 
    def loss_fn(self, lbl, y):
        """ loss function between true labels lbl and prediction y """
        criterion  = gluon.loss.L2Loss()
        criterion2 = gluon.loss.SigmoidBinaryCrossEntropyLoss()
        veci = 5. * nd.array(lbl[:,1:], ctx=self.device)
        lbl  = nd.array(lbl[:,0]>.5, ctx=self.device)
        loss = criterion(y[:,:-1] , veci) + criterion2(y[:,-1] , lbl)
        return loss
```

found at https://github.com/MouseLand/cellpose/blob/4ac3858b48a5d049d81f412055d6d2c2dab4393c/cellpose/models.py#L1257


I interpret it as this:

the variable veci will consist of the first two channels of the image (Xflow and Yflow) multiplied by 5 (I don't know why)

and lbl will consist of the third channel of the image (cellprobability channel), which I think is an image with probability values for the pixel belonging to a cell or not. If the probability is higher than 0.5, it will be considered a pixel belonging to a cell (True).

Then two separate loss functions are applied, L2Loss for veci (Xflow and Yflow channel), and SigmoidBinaryCrossEntropyLoss for the cellprobability channel. 


I tried to convert this to work on our model, and using Keras. I did like this:


```python 
def loss_fn(y, y_pred):
    
    criterion1 = tf.keras.losses.MeanSquaredError()
    criterion2 = tf.keras.losses.BinaryCrossentropy()
    lab_ch = y_pred[:,:,2]
    flow_ch = y_pred[:,:,0:2]
    loss = criterion1(y[:,:,0:2], flow_ch) + criterion2(y[:,:,2], lab_ch)
    
    return loss
```

I wanted to set ```lab_ch = y_pred[:,:,2]>0.5``` but it was then converted to a bool object which caused error, and I am lost in how I would implement the threshold to this function. I got a model to run with this settings, but the predictions did not give any good results. They looked like this:





images show:
1. Predicted image
2. Image it was predicted from

## <center>Prediction cell-model 17</center>

<center>Predicted image </center> | <center>Image it was predicted from</center>
- | - 
<img src="images_jupyter/prob_maps/MFGTMPcx7_170702090001_B22f15d1.png">| <img src="images_jupyter/norm_images/MFGTMPcx7_170702090001_B22f15d1.png">


I have not done any other tweaks with the model, I thought that if I manage to implement a loss function which seem to work, I would continue to tweak the model and doing other changes.

The cellpose model is described under methods in their article. Perhaps the Unet also needs to be changed to get reasonable results?


#### Creating-flow-masks.py

```python

##################################### Cellpose script to create masks #############################################


import numpy as np
import skimage
import skimage.io
import os
import scipy
import scipy.ndimage
from skimage.color import rgb2lab
from skimage import img_as_ubyte
import skimage.morphology
from tqdm import trange

#utils
def diameters(masks):
    """ get median 'diameter' of masks """
    _, counts = np.unique(np.int32(masks), return_counts=True)
    counts = counts[1:]
    md = np.median(counts**0.5)
    if np.isnan(md):
        md = 0
    md /= (np.pi**0.5)/2
    return md, counts**0.5

def _extend_centers(T,y,x,ymed,xmed,Lx, niter):
    """ run diffusion from center of mask (ymed, xmed) on mask pixels (y, x)
    Parameters
    --------------
    T: float64, array
        _ x Lx array that diffusion is run in
    y: int32, array
        pixels in y inside mask
    x: int32, array
        pixels in x inside mask
    ymed: int32
        center of mask in y
    xmed: int32
        center of mask in x
    Lx: int32
        size of x-dimension of masks
    niter: int32
        number of iterations to run diffusion
    Returns
    ---------------
    T: float64, array
        amount of diffused particles at each pixel
    """

    for t in range(niter):
        T[ymed*Lx + xmed] += 1
        T[y*Lx + x] = 1/9. * (T[y*Lx + x] + T[(y-1)*Lx + x]   + T[(y+1)*Lx + x] +
                                            T[y*Lx + x-1]     + T[y*Lx + x+1] +
                                            T[(y-1)*Lx + x-1] + T[(y-1)*Lx + x+1] +
                                            T[(y+1)*Lx + x-1] + T[(y+1)*Lx + x+1])
    return T



def masks_to_flows(masks):
    """ convert masks to flows using diffusion from center pixel
    Center of masks where diffusion starts is defined to be the 
    closest pixel to the median of all pixels that is inside the 
    mask. Result of diffusion is converted into flows by computing
    the gradients of the diffusion density map. 
    Parameters
    -------------
    masks: int, 2D or 3D array
        labelled masks 0=NO masks; 1,2,...=mask labels
    Returns
    -------------
    mu: float, 3D or 4D array 
        flows in Y = mu[-2], flows in X = mu[-1].
        if masks are 3D, flows in Z = mu[0].
    mu_c: float, 2D or 3D array
        for each pixel, the distance to the center of the mask 
        in which it resides 
    """
    #if len(masks.shape) >2:
    #    masks = rgb2lab(masks)
    #    masks = masks[:,:,0]
    #masks = skimage.morphology.label(masks)
    #print(masks.shape)
    if masks.ndim > 2:
        Lz, Ly, Lx = masks.shape
        mu = np.zeros((3, Lz, Ly, Lx), np.float32)
        for z in range(Lz):
            mu0 = masks_to_flows(masks[z])[0]
            mu[[1,2], z] += mu0
        for y in range(Ly):
            mu0 = masks_to_flows(masks[:,y])[0]
            mu[[0,2], :, y] += mu0
        for x in range(Lx):
            mu0 = masks_to_flows(masks[:,:,x])[0]
            mu[[0,1], :, :, x] += mu0
        return mu, None

    Ly, Lx = masks.shape
    mu = np.zeros((2, Ly, Lx), np.float64)
    mu_c = np.zeros((Ly, Lx), np.float64)
    
    nmask = masks.max()
    slices = scipy.ndimage.find_objects(masks)
    dia = diameters(masks)[0]
    s2 = (.15 * dia)**2
    for i,si in enumerate(slices):
        if si is not None:
            sr,sc = si
            ly, lx = sr.stop - sr.start + 1, sc.stop - sc.start + 1
            y,x = np.nonzero(masks[sr, sc] == (i+1))
            y = y.astype(np.int32) + 1
            x = x.astype(np.int32) + 1
            ymed = np.median(y)
            xmed = np.median(x)
            imin = np.argmin((x-xmed)**2 + (y-ymed)**2)
            xmed = x[imin]
            ymed = y[imin]

            d2 = (x-xmed)**2 + (y-ymed)**2
            mu_c[sr.start+y-1, sc.start+x-1] = np.exp(-d2/s2)

            niter = 2*np.int32(np.ptp(x) + np.ptp(y))
            T = np.zeros((ly+2)*(lx+2), np.float64)
            T = _extend_centers(T, y, x, ymed, xmed, np.int32(lx), niter)
            T[(y+1)*lx + x+1] = np.log(1.+T[(y+1)*lx + x+1])

            dy = T[(y+1)*lx + x] - T[(y-1)*lx + x]
            dx = T[y*lx + x+1] - T[y*lx + x-1]
            mu[:, sr.start+y-1, sc.start+x-1] = np.stack((dy,dx))

    mu /= (1e-20 + (mu**2).sum(axis=0)**0.5)

    return mu, mu_c


def labels_to_flows(labels, files=None):
    """ convert labels (list of masks or flows) to flows for training model 
    if files is not None, flows are saved to files to be reused
    Parameters
    --------------
    labels: list of ND-arrays
        labels[k] can be 2D or 3D, if [3 x Ly x Lx] then it is assumed that flows were precomputed.
        Otherwise labels[k][0] or labels[k] (if 2D) is used to create flows and cell probabilities.
    Returns
    --------------
    flows: list of [4 x Ly x Lx] arrays
        flows[k][0] is labels[k], flows[k][1] is cell probability, flows[k][2] is Y flow, and flows[k][3] is X flow
    """
    
    nimg = len(labels)
    if labels[0].ndim < 3:
        labels = [labels[n][np.newaxis,:,:] for n in range(nimg)]

    if labels[0].shape[0] == 1 or labels[0].ndim < 3:
        print('NOTE: computing flows for labels (could be done before to save time)')
        # compute flows        
        veci = []
            
        veci = [masks_to_flows(labels[n][0])[0] for n in trange(nimg)]
        #concatenate flows with cell probability
        flows = [np.concatenate((labels[n][[0]], labels[n][[0]]>0.5, veci[n], labels[n]), axis=0).astype(np.float32)
                    for n in range(nimg)]
        if files is not None:
            for flow, file in zip(flows, files):
                file_name = os.path.splitext(file)[0]
                tifffile.imsave(file_name+'_flows.tif', flow)
    else:
        print('flows precomputed')
        flows = [labels[n].astype(np.float32) for n in range(nimg)]
    return flows


################################################ Addition by Malou ##############################################

## Path to where the raw annotations are located.
file_path = "/media/malou/Seagate Expansion Drive/Malou_Master/Master/data/Annotationsets/Cell-annotationset-1and2/"


# Load all images to list to use in flows_to_labels
filelist = os.listdir(file_path)
save_path = file_path + "flows/"
imlist = []
imname = []
for file in filelist:
    if file.endswith(".png"):
        imname.append(file)
        im = skimage.io.imread(file_path + file)
        im = rgb2lab(im)
        im = im[:,:,0]
        im = skimage.morphology.label(im)
        imlist.append(im)


# Creating flow masks
flows = labels_to_flows(imlist)



cellprob_im = []
Yflow = []
Xflow = []

# Create lists of flows_to_labels output (indexing described in function)
for i in range(len(flows)):

    cellprob_im.append(flows[i][1])
    Yflow.append(flows[i][2])
    Xflow.append(flows[i][3])

    
# Converting lists to array to stack them later
cellprob_im = np.asarray(cellprob_im)
Yflow = np.asarray(Yflow)
Xflow = np.asarray(Xflow)

# Currently ranges from -1 to 1, change to 0-255 8bit
cellprob_im = img_as_ubyte(cellprob_im)
Yflow = img_as_ubyte(Yflow)
Xflow = img_as_ubyte(Xflow)
for i in range(len(cellprob_im)):

    merged = np.dstack((Xflow[i],Yflow[i],cellprob_im[i])) # Merging output from flows_to_labels
    skimage.io.imsave(save_path + imname[i], merged)
    
    
```